# Imports

In [174]:
import pandas as pd
import numpy as np
import spreg
import libpysal
import geopandas as gpd
from libpysal.weights import Queen
from spreg import ML_Lag

# Data

In [188]:
# Read in the data
# db = libpysal.io.open("../_data/shapefile/map_municipalities.dbf","r")
db = pd.read_csv('../_data/data_municipalities_final.csv')
db["partition_austrian"] = db["partition_austrian"].astype(int)
db["partition_prussian"] = db["partition_prussian"].astype(int)
db["partition_russian"] = db["partition_russian"].astype(int)
# Train and test datasets (test on Wielkopolskie)
db["municipality_code_str"] = db["municipality_code"].astype(str)
db.loc[:607, 'municipality_code_str'] = '0' + db.loc[:607, 'municipality_code'].astype(str)
db_train = db[db["municipality_code_str"].str[0:2] != "30"]
db_test = db[db["municipality_code_str"].str[0:2] == "30"]
# drop
db_train.drop(columns=["municipality_code_str"], inplace=True)
db_test.drop(columns=["municipality_code_str"], inplace=True)
print(db_train.shape, db_test.shape)
# Specify the path to the shapefile
shapefile_path = '../_data/shapefile/map_municipalities.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Train and test datasets
gdf_train = gdf[gdf["mncplty_c"].str[0:2] != "30"]
gdf_test = gdf[gdf["mncplty_c"].str[0:2] == "30"]
print(gdf_train.shape, gdf_test.shape)

/tmp/ipykernel_343002/887902822.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/887902822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)


(2251, 48) (226, 48)
(2251, 3) (226, 3)


# SAR Model

## On train dataset

In [289]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()
x_names.remove("percent_vaccinated")
x_names.remove("percent_vaccinated_log")
x_names.remove("county_code")
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x_names.remove("municipality_code")
x_names.remove("municipality_name")
# General to specific model selection
x_names.remove("population_total_f")
x_names.remove("partition_austrian")
x_names.remove("appartments_per_1000_persons")
x_names.remove("doctors_per_1000_persons")
x_names.remove("average_wage_relative")
x_names.remove("revenues_per_capita_CIT")
x_names.remove("revenues_per_capita")
x_names.remove("population_total")
x_names.remove("partition_russian")
x_names.remove("population_total_m")
x_names.remove("persons_per_library")
x_names.remove("KONFEDERACJA_party")
x_names.remove("population_per_pharmacy")
x_names.remove("population_density")
x_names.remove("healthcare_advices")
x_names.remove("unemployment_rate_m")
x_names.remove("education_share_primary")
x_names.remove("education_share_higher")
x_names.remove("unemployment_rate")
x_names.remove("partition_prussian")
x_names.remove("percent_population_below_25")
x_names.remove("percent_population_25_60")
x_names.remove("area_km2")
x_names.remove("education_share_vocational")
x = db_train[x_names].values
w_name = "border_contiguity"
w.transform = 'r'
mllag = ML_Lag(y,x,w,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True) 


/tmp/ipykernel_343002/4018778415.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


In [290]:
print(mllag.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          18
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2233
Pseudo R-squared    :      0.8244
Spatial Pseudo R-squared:  0.6802
Log likelihood      :   1233.2233
Sigma-square ML     :      0.0179                Akaike info criterion :   -2430.447
S.E of regression   :      0.1340                Schwarz criterion     :   -2327.502

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
----------------------------------------

### RMSE in-sample

In [272]:
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
rmse = np.sqrt(np.sum((y - mllag.predy)**2)/len(y))
print(rmse)

0.13222419593118737


## On test dataset

In [ ]:
# firguring out betas and rho
# first beta is intercept
# last beta is rho
mllag.betas, mllag.rho

In [236]:
# Predict on the test dataset
# Prepare y_test
y_test = db_test["percent_vaccinated_log"].values
# Create a spatial weights object
w = Queen.from_dataframe(gdf_test)
# Calcualte Wy
w_array = w.to_sparse().toarray()
row_sums = w_array.sum(axis=1)
w_array_normalized = w_array / row_sums[:, np.newaxis]
Wy = np.dot(w_array_normalized, y_test)
# Calculate y_pred
y_pred = mllag.betas[0] + np.dot(db_test[x_names].values,mllag.betas[1:-1]) + mllag.betas[-1]*Wy
# Calculate RMSE
rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
print(rmse)

3.6441942048498843


/tmp/ipykernel_343002/895585023.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


## leave one out for other voivoidships

In [346]:
# Read in the data
# db = libpysal.io.open("../_data/shapefile/map_municipalities.dbf","r")
db = pd.read_csv('../_data/data_municipalities_final.csv')
db["partition_austrian"] = db["partition_austrian"].astype(int)
db["partition_prussian"] = db["partition_prussian"].astype(int)
db["partition_russian"] = db["partition_russian"].astype(int)
db["municipality_code_str"] = db["municipality_code"].astype(str)
db.loc[:607, 'municipality_code_str'] = '0' + db.loc[:607, 'municipality_code'].astype(str)
# Specify the path to the shapefile
shapefile_path = '../_data/shapefile/map_municipalities.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# chosen variables
predictors = ['urbanization_rate',
                'installations_network_gas',
                'library_readers_per_1000_persons',
                'forests_area',
                'unemployment_rate_f',
                'revenues_per_capita_PIT',
                'net_scholarization',
                'education_share_secondary',
                'tourits_per_1000_persons',
                'percent_population_over_60',
                'election_presence',
                'percent_invalid_votes',
                'PIS_party',
                'KO_party',
                'PSL_party',
                'SLD_party']

# list of models
SAR_models = []
in_sample_rmse = []
out_of_sample_rmse = []

for voivodeship_code in ["02", "04", "06", "08", "10", "12", "14", "16", "18", "20", "22", "24", "26", "28", "30","32"]:
    print(f"current voivodeship: {voivodeship_code}")
    # Train and test datasets 
    db_train = db[db["municipality_code_str"].str[0:2] != voivodeship_code]
    db_test = db[db["municipality_code_str"].str[0:2] == voivodeship_code]
    # drop
    db_train.drop(columns=["municipality_code_str"], inplace=True)
    db_test.drop(columns=["municipality_code_str"], inplace=True)
    print("db",db_train.shape, db_test.shape)

    # Train and test datasets
    gdf_train = gdf[gdf["mncplty_c"].str[0:2] != voivodeship_code]
    gdf_test = gdf[gdf["mncplty_c"].str[0:2] == voivodeship_code]
    print("gdf", gdf_train.shape, gdf_test.shape)

    # Create a spatial weights object
    w = Queen.from_dataframe(gdf_train)
    ds_name = "municipalities_train"
    y_name = "percent_vaccinated_log"
    y = db_train[y_name].values
    y.shape = (len(y),1)
    x_names = predictors
    x = db_train[x_names].values
    w_name = "border_contiguity"
    w.transform = 'r'
    mllag = ML_Lag(y,x,w,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True)
    # append model
    SAR_models.append(mllag)
    # append in-sample RMSE
    y = db_train[y_name].values
    rmse = np.sqrt(np.sum((y - mllag.predy)**2)/len(y))
    in_sample_rmse.append(rmse)
    # append out-of-sample RMSE
    # Prepare y_test
    y_test = db_test["percent_vaccinated_log"].values
    # Create a spatial weights object
    w = Queen.from_dataframe(gdf_test)
    # Calcualte Wy
    w_array = w.to_sparse().toarray()
    row_sums = w_array.sum(axis=1)
    w_array_normalized = w_array / row_sums[:, np.newaxis]
    Wy = np.dot(w_array_normalized, y_test)
    # Calculate y_pred
    y_pred = mllag.betas[0] + np.dot(db_test[x_names].values,mllag.betas[1:-1]) + mllag.betas[-1]*Wy
    # Calculate RMSE
    rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
    out_of_sample_rmse.append(rmse)

    

current voivodeship: 02
db (2308, 48) (169, 48)
gdf (2308, 3) (169, 3)


/tmp/ipykernel_343002/1113939114.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:54: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


current voivodeship: 04
db (2333, 48) (144, 48)
gdf (2333, 3) (144, 3)


/tmp/ipykernel_343002/1113939114.py:74: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)
/tmp/ipykernel_343002/1113939114.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:54: FutureWarning: `use_index` defaults to False but will def

/tmp/ipykernel_343002/1113939114.py:74: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


...

current voivodeship: 30
db (2251, 48) (226, 48)
gdf (2251, 3) (226, 3)


/tmp/ipykernel_343002/1113939114.py:74: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)
/tmp/ipykernel_343002/1113939114.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:54: FutureWarning: `use_index` defaults to False but will def

current voivodeship: 32
db (2364, 48) (113, 48)
gdf (2364, 3) (113, 3)


/tmp/ipykernel_343002/1113939114.py:74: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)
/tmp/ipykernel_343002/1113939114.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/1113939114.py:54: FutureWarning: `use_index` defaults to False but will def

/tmp/ipykernel_343002/1113939114.py:74: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


In [347]:
# save lists to gzip pickle
import gzip
import pickle
with gzip.open('SAR_models.pkl', 'wb') as f:
    pickle.dump(SAR_models, f)
with gzip.open('SAR_in_sample_rmse.pkl', 'wb') as f:
    pickle.dump(in_sample_rmse, f)
with gzip.open('SAR_out_of_sample_rmse.pkl', 'wb') as f:
    pickle.dump(out_of_sample_rmse, f)


In [342]:
mllag.name_x

['CONSTANT',
 'urbanization_rate',
 'installations_network_gas',
 'library_readers_per_1000_persons',
 'forests_area',
 'unemployment_rate_f',
 'revenues_per_capita_PIT',
 'net_scholarization',
 'education_share_secondary',
 'tourits_per_1000_persons',
 'percent_population_over_60',
 'election_presence',
 'percent_invalid_votes',
 'PIS_party',
 'KO_party',
 'PSL_party',
 'SLD_party',
 'W_percent_vaccinated_log']

# SDM

## In-sample

In [338]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()
x_names.remove("percent_vaccinated")
x_names.remove("percent_vaccinated_log")
x_names.remove("county_code")
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x_names.remove("municipality_code")
x_names.remove("municipality_name")
# General to specific model selection
x_names.remove("population_total_f")
x_names.remove("partition_austrian")
x_names.remove("KONFEDERACJA_party")
x_names.remove("education_share_primary")
x_names.remove("percent_population_below_25")
x_names.remove("persons_per_library")
x_names.remove("revenues_per_capita_CIT")
x_names.remove("revenues_per_capita")
x_names.remove("appartments_per_1000_persons")
x_names.remove("education_share_higher")
x_names.remove("average_wage_relative")
x_names.remove("unemployment_rate_m")
x_names.remove("education_share_secondary")
x_names.remove("education_share_vocational")
x_names.remove("population_total_m")
x_names.remove("population_per_pharmacy")
x_names.remove("population_total")
x_names.remove("unemployment_rate")
x_names.remove("unemployment_rate_f")
x_names.remove("doctors_per_1000_persons")
x_names.remove("partition_prussian")
x_names.remove("tourits_per_1000_persons")
x_names.remove("PIS_party")
x_names.remove("partition_russian")
x_names.remove("urbanization_rate")
x = db_train[x_names].values
w_name = "border_contiguity"
w.transform = 'r'
mllag_SDM = ML_Lag(y,x,w,slx_lags=1,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True) 


/tmp/ipykernel_343002/163918878.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


In [335]:
print(mllag_SDM.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG WITH SLX - SPATIAL DURBIN MODEL (METHOD = FULL)
-------------------------------------------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          32
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2219
Pseudo R-squared    :      0.8529
Spatial Pseudo R-squared:  0.7237
Log likelihood      :   1378.2272
Sigma-square ML     :      0.0151                Akaike info criterion :   -2692.454
S.E of regression   :      0.1228                Schwarz criterion     :   -2509.442

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-St

## RMSE in-sample

In [336]:
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
rmse = np.sqrt(np.sum((y - mllag_SDM.predy)**2)/len(y))
print(rmse)

0.12278878733040159


## On test dataset

In [341]:
# Predict on the test dataset
# Prepare y_test
y_test = db_test["percent_vaccinated_log"].values
# Create a spatial weights object
w = Queen.from_dataframe(gdf_test)
# Calcualte Wy
w_array = w.to_sparse().toarray()
row_sums = w_array.sum(axis=1)
w_array_normalized = w_array / row_sums[:, np.newaxis]
Wy = np.dot(w_array_normalized, y_test)
Wx = np.dot(w_array_normalized, db_test[x_names].values)
# Calculate y_pred
y_pred = mllag_SDM.betas[0] + np.dot(db_test[x_names].values,mllag_SDM.betas[1:-1][:15]) + mllag_SDM.betas[-1]*Wy + np.dot(Wx, mllag_SDM.betas[1:-1][15:])
# Calculate RMSE
rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
print(rmse)

3.56667798949035


/tmp/ipykernel_343002/1902786663.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


## Leave one out on all voivoidships

In [348]:
# Read in the data
# db = libpysal.io.open("../_data/shapefile/map_municipalities.dbf","r")
db = pd.read_csv('../_data/data_municipalities_final.csv')
db["partition_austrian"] = db["partition_austrian"].astype(int)
db["partition_prussian"] = db["partition_prussian"].astype(int)
db["partition_russian"] = db["partition_russian"].astype(int)
db["municipality_code_str"] = db["municipality_code"].astype(str)
db.loc[:607, 'municipality_code_str'] = '0' + db.loc[:607, 'municipality_code'].astype(str)
# Specify the path to the shapefile
shapefile_path = '../_data/shapefile/map_municipalities.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# chosen variables
predictors = ["area_km2",
                "population_density",
                "healthcare_advices",
                "installations_network_gas",
                "library_readers_per_1000_persons",
                "forests_area",
                "revenues_per_capita_PIT",
                "net_scholarization",
                "percent_population_25_60",
                "percent_population_over_60",
                "election_presence",
                "percent_invalid_votes",
                "KO_party",
                "PSL_party",
                "SLD_party"]



# list of models
SDM_models = []
in_sample_rmse = []
out_of_sample_rmse = []

for voivodeship_code in ["02", "04", "06", "08", "10", "12", "14", "16", "18", "20", "22", "24", "26", "28", "30","32"]:
    print(f"current voivodeship: {voivodeship_code}")
    # Train and test datasets 
    db_train = db[db["municipality_code_str"].str[0:2] != voivodeship_code]
    db_test = db[db["municipality_code_str"].str[0:2] == voivodeship_code]
    # drop
    db_train.drop(columns=["municipality_code_str"], inplace=True)
    db_test.drop(columns=["municipality_code_str"], inplace=True)
    print("db",db_train.shape, db_test.shape)

    # Train and test datasets
    gdf_train = gdf[gdf["mncplty_c"].str[0:2] != voivodeship_code]
    gdf_test = gdf[gdf["mncplty_c"].str[0:2] == voivodeship_code]
    print("gdf", gdf_train.shape, gdf_test.shape)

    # Create a spatial weights object
    w = Queen.from_dataframe(gdf_train)
    ds_name = "municipalities_train"
    y_name = "percent_vaccinated_log"
    y = db_train[y_name].values
    y.shape = (len(y),1)
    x_names = predictors
    x = db_train[x_names].values
    w_name = "border_contiguity"
    w.transform = 'r'
    mllag = ML_Lag(y,x,w,slx_lags=1, name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True)
    # append model
    SDM_models.append(mllag)
    # append in-sample RMSE
    y = db_train[y_name].values
    rmse = np.sqrt(np.sum((y - mllag.predy)**2)/len(y))
    in_sample_rmse.append(rmse)
    # append out-of-sample RMSE
    # Prepare y_test
    y_test = db_test["percent_vaccinated_log"].values
    # Create a spatial weights object
    w = Queen.from_dataframe(gdf_test)
    # Calcualte Wy and Wx
    w_array = w.to_sparse().toarray()
    row_sums = w_array.sum(axis=1)
    w_array_normalized = w_array / row_sums[:, np.newaxis]
    Wy = np.dot(w_array_normalized, y_test)
    Wx = np.dot(w_array_normalized, db_test[x_names].values)
    # Calculate y_pred
    y_pred = mllag.betas[0] + np.dot(db_test[x_names].values,mllag.betas[1:-1][:15]) + mllag.betas[-1]*Wy + np.dot(Wx, mllag.betas[1:-1][15:])
    # Calculate RMSE
    rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
    out_of_sample_rmse.append(rmse)
    
    

current voivodeship: 02
db (2308, 48) (169, 48)
gdf (2308, 3) (169, 3)


/tmp/ipykernel_343002/824467191.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:55: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


current voivodeship: 04
db (2333, 48) (144, 48)
gdf (2333, 3) (144, 3)


/tmp/ipykernel_343002/824467191.py:75: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)
/tmp/ipykernel_343002/824467191.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:55: FutureWarning: `use_index` defaults to False but will default

current voivodeship: 06
db (2264, 48) (213, 48)
gdf (2264, 3) (213, 3)


...

current voivodeship: 30
db (2251, 48) (226, 48)
gdf (2251, 3) (226, 3)


/tmp/ipykernel_343002/824467191.py:75: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)
/tmp/ipykernel_343002/824467191.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:55: FutureWarning: `use_index` defaults to False but will default

current voivodeship: 32
db (2364, 48) (113, 48)
gdf (2364, 3) (113, 3)


/tmp/ipykernel_343002/824467191.py:75: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)
/tmp/ipykernel_343002/824467191.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/824467191.py:55: FutureWarning: `use_index` defaults to False but will default

/tmp/ipykernel_343002/824467191.py:75: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


In [349]:
# save lists to gzip pickle
import gzip
import pickle
with gzip.open('SDM_models.pkl', 'wb') as f:
    pickle.dump(SDM_models, f)
with gzip.open('SDM_in_sample_rmse.pkl', 'wb') as f:
    pickle.dump(in_sample_rmse, f)
with gzip.open('SDM_out_of_sample_rmse.pkl', 'wb') as f:
    pickle.dump(out_of_sample_rmse, f)


# Table with results

In [356]:
# Load SAR model RMSE values
with gzip.open('SAR_in_sample_rmse.pkl', 'rb') as file:
    sar_in_sample_rmse = pickle.load(file)
with gzip.open('SAR_out_of_sample_rmse.pkl', 'rb') as file:
    sar_out_of_sample_rmse = pickle.load(file)

# Load SDM model RMSE values
with gzip.open('SDM_in_sample_rmse.pkl', 'rb') as file:
    sdm_in_sample_rmse = pickle.load(file)
with gzip.open('SDM_out_of_sample_rmse.pkl', 'rb') as file:
    sdm_out_of_sample_rmse = pickle.load(file)

sar_in_sample_rmse, sar_out_of_sample_rmse, sdm_in_sample_rmse, sdm_out_of_sample_rmse

# Voivodeships
voivodeships = ["Dolnośląskie", "Kujawsko-Pomorskie", "Lubelskie", "Lubuskie", "Łódzkie", "Małopolskie", 
                "Mazowieckie", "Opolskie", "Podkarpackie", "Podlaskie", "Pomorskie", "Śląskie", "Świętokrzyskie", 
                "Warmińsko-Mazurskie", "Wielkopolskie", "Zachodniopomorskie"]

# Create a DataFrame
df = pd.DataFrame({"Voivodeship": voivodeships, 
                   "SAR_in_sample_rmse": sar_in_sample_rmse,
                   "SAR_out_of_sample_rmse": sar_out_of_sample_rmse,
                   "SDM_in_sample_rmse": sdm_in_sample_rmse,
                   "SDM_out_of_sample_rmse": sdm_out_of_sample_rmse})
df.to_csv("SAR_SDM_model_comparison.csv", index=False)
df.to_latex("SAR_SDM_model_comparison.tex", index=False)